In [1]:
from human import *
import pandas as pd
from query_generation import *
from sampling import *
import gym
import pickle

In [2]:
env = gym.make('LunarLander-v2')

In [3]:
df = pd.DataFrame(columns=["run #", "pref_iter", "type", "value"])

In [4]:
HUMAN_TYPE = "term"

SIM_ITER_COUNT = 100
N_PREF_ITERS = 25
N_ITERS_EXP = 8

N_QUERY = 2
INC_PREV_QUERY = False
UPDATE_FUNC = "approx"
EPSILON = 0
N_SAMPLES_SUMM = 50000
N_SAMPLES_EXP = N_SAMPLES_SUMM

BETA_DEMO = 0.1
BETA_PREF = 5
BETA_HUMAN = 1

In [5]:
demo_files = ['1669268176.6269968',
 '1669268196.0336297',
 '1669268212.2845922',
 '1669268231.2647314',
 '1669268269.2316186',
 '1669268294.8198466',
 '1669268315.7406812',
 '1669268331.8071287',
 '1669268365.5589268',
 '1669268383.076348']

In [6]:
# DOM = domain.LunarLander(time_steps=150)
UPDATE_FUNC = "pick_best"
TRUE_WEIGHT = [-0.4, 0.4, -0.2, -0.7]
GEN_DEMOS = False
demo_path = 'data/demonstrations/gym-'
DEMOS = [pickle.load(open(demo_path+ f'{demo_name}' + '.pickle', 'rb'), encoding='latin1')[1] for demo_name in demo_files]
QUERY_LENGTH = 10
ITER_COUNT = 2

In [7]:
qg = ApproxQueryGenerator(env=env, num_queries=N_QUERY, query_length=QUERY_LENGTH,
                                                       num_expectation_samples=N_SAMPLES_EXP,
                                                       include_previous_query=INC_PREV_QUERY,
                                                       generate_scenario=GEN_DEMOS, update_func=UPDATE_FUNC,
                                                       beta_pref=BETA_PREF)

In [8]:
H = TerminalHuman(env, UPDATE_FUNC)

### Testing the watch function

In [11]:
controls = np.random.choice([0, 1, 2, 3], size=(300))

In [12]:
features = watch(env, controls)

81: -119.85908762950282
Finished running game in 0.014091253280639648s


### back to dempref

In [9]:
for i in range(ITER_COUNT):
    ### Processing demonstrations
    sampler = Sampler(n_query=N_QUERY, dim_features=4,
                                update_func=UPDATE_FUNC,
                                beta_demo=BETA_DEMO, beta_pref=BETA_PREF)
    if len(DEMOS) > 0:
        phi_demos = [np_features(x, True) for x in DEMOS]
        sampler.load_demo(np.array(phi_demos))
        cleaned_demos = DEMOS

    ## Computing initial estimates
    samples = sampler.sample(N=N_SAMPLES_SUMM)
    mean_w = np.mean(samples, axis=0)
    mean_w = mean_w / np.linalg.norm(mean_w)
    var_w = np.var(samples, axis=0)
    data = [[i+1, 0, "mean", mean_w],
            [i+1, 0, "var", var_w]]
    print("Estimate of w: " + str(mean_w)) # TODO: Add different levels of verbose mode
    print("Estimate of variance: " + str(sum(var_w)))
    # computing convergence measure if we are in simulation
    
    df = df.append(pd.DataFrame(data, columns=["run #", "pref_iter", "type", "value"]), ignore_index=True)

    ### Preferences loop
    for j in range(N_PREF_ITERS):
        print("\n\n*** Preferences Loop %d\n" % (j))

        ## Generate queries while ensuring that features of queries are epsilon apart
        query_diff = 0
        print("Generating queries")
        while query_diff <= EPSILON:
            queries = qg.queries(samples)
            query_diffs = []
            for m in range(len(queries)):
                for n in range(m):
                    query_diffs.append(np.linalg.norm(np_features(queries[m], True) - np_features(queries[n], True)))
            query_diff = max(query_diffs)
            print(f"max diff: {query_diff} epsilon: {EPSILON}")

        ## Querying human
        print('\a')
        rank = H.input(queries)

        ## Creating dictionary mapping rankings to features of queries and loading into sampler
        features = [np_features(x, True) for x in queries]
        phi = {k : features[k] for k in range(len(queries))}
        sampler.load_prefs(phi, rank)

        ## Recording data from this run
        samples = sampler.sample(N=N_SAMPLES_SUMM)
        mean_w = np.mean(samples, axis=0)
        mean_w = mean_w / np.linalg.norm(mean_w)
        var_w = np.var(samples, axis=0)
        data = [[i+1, j+1, "mean", mean_w],
                [i+1, j+1, "var", var_w]]
        print("Estimate of w: " + str(mean_w))
        print("Estimate of variance: " + str(sum(var_w)))
        # if self.human_type != "term":
        #     m = np.mean(
        #         [np.dot(w, self.true_weight) / np.linalg.norm(w) / np.linalg.norm(self.true_weight) for w in
        #             samples])
        #     data.append([i+1, j+1, "m", m])
        #     print("Estimate of m: " + str(m) + "\n\n")
        df = df.append(pd.DataFrame(data, columns=["run #", "pref_iter", "type", "value"]), ignore_index=True)
    ## Resetting for next run
    # sampler.clear_pref()
    # if self.inc_prev_query and self.n_demos > 0:
    #     last_query_picked = [d for d in cleaned_demos]


Finished constructing sampling function in 1.2857749462127686 seconds
Estimate of w: [-0.85145178  0.00284176 -0.07647409  0.51881934]
Estimate of variance: 0.9999590810004431


*** Preferences Loop 0

Generating queries
2
2
2
2
2
2
2
2
2
2
2
2
2
2


KeyboardInterrupt: 

In [16]:
env.action_space.sample()

0

In [10]:
!export DISPLAY=:0

In [27]:
[(env.observation_space.low[i], env.observation_space.high[i]) for i in range(env.observation_space.shape[0])]

[(-inf, inf),
 (-inf, inf),
 (-inf, inf),
 (-inf, inf),
 (-inf, inf),
 (-inf, inf),
 (-inf, inf),
 (-inf, inf)]

In [28]:
env.action_space.shape[0]

IndexError: tuple index out of range